# Extract Factors
Takes the `points_table.json` file as an input and computes the a best fit quadratic for each event mapping time or performance to the number of points and saving the fit parameters along with the event info in `fit_params.json`

In [3]:
import json
import numpy as np

def fit_function(x, a, ref, shift=0):
    return a*(x + ref)**2 + shift

def loss(params, x_data, y_data):
    a, ref, shift = params
    y_pred = fit_function(x_data, a, ref, shift)
    return np.sum(np.abs(np.floor(y_pred) - np.floor(y_data)))

def time_to_seconds(time_str):
    parts = time_str.split(':')
    seconds = float(parts[-1])
    for i, part in enumerate(reversed(parts[:-1])):
        seconds += int(part) * 60 ** (i+1)
    if '.' not in time_str:
        return seconds + 0.5
    else:
        return seconds + 0.005

points_table = json.load(open("2025_points_table.json", "r"))
points = np.array(points_table["Points"], dtype=float) + 0.5

def get_fit(gender, event):
    times = np.array(points_table[gender][event]['Scores'])
    times = np.array([time_to_seconds(t) if t != "-" else np.nan for t in times])
    print(f"{event}: {times[~np.isnan(times)][-1]}")
    # curve fitting
    x_data = times[~np.isnan(times)]
    y_data = points[~np.isnan(times)]
    a, b, c = np.polyfit(x_data, y_data, 2)
    ref = b / (2 * a)
    shift = c - b**2 / (4 * a)
    # calculate the loss
    error = loss([a, ref, shift], x_data, y_data)
    print(f"{event}: a={a}, ref={ref}, shift={shift}, error={error}")
    return a, ref, shift

In [4]:
fit_params = {"Men": {},
              "Women": {}}

for gender in fit_params.keys():
    for event in points_table[gender].keys():
        fits = get_fit(gender, event)
        fit_params[gender][event] = {}
        fit_params[gender][event]["Fits"] = fits
        fit_params[gender][event]["Distance"] = points_table[gender][event]['Distance']
        fit_params[gender][event]["Category"] = points_table[gender][event]['Category']
        fit_params[gender][event]["Surface"] = points_table[gender][event]['Surface']
        fit_params[gender][event]["Mixed"] = points_table[gender][event]['Mixed']

# save the fit parameters to a JSON file
with open("2025_fit_params.json", "w") as f:
    json.dump(fit_params, f, indent=4)


50m: 9.095
50m: a=95.82235386298277, ref=-9.204401087362431, shift=0.12340216836128093, error=6.0
55m: 9.885000000000002
55m: a=78.92276088460528, ref=-10.004168296259685, shift=0.1737406783249753, error=5.0
60m: 10.575000000000001
60m: a=68.62032200466454, ref=-10.70428001635122, shift=0.14057243182287493, error=12.0
100m: 16.794999999999998
100m: a=24.64221166330623, ref=-17.00253155827432, shift=0.28133795201483736, error=12.0
200m: 35.055
200m: a=5.083329625707246, ref=-35.49611159736069, shift=0.5957410214796255, error=12.0
200m sh: 35.555
200m sh: a=5.042898433343425, ref=-35.99671911026435, shift=0.5826109375357191, error=22.0
300m: 56.465
300m: a=1.82965702474357, ref=-57.2026881433823, shift=0.44464140746458725, error=9.0
300m sh: 57.255
300m sh: a=1.8029639297594922, ref=-58.0004364259106, shift=0.48213317584577453, error=0.0
400m: 78.015
400m: a=1.0210130425533055, ref=-78.99969305639164, shift=0.5029880051552027, error=0.0
400m sh: 79.595
400m sh: a=0.9810285010269529, ref=